In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from collections import defaultdict
from pprint import pprint

# 創建一個 Options 物件
chrome_options = Options()

chrome_options.add_argument("--start-maximized") # Chrome 瀏覽器在啟動時最大化視窗
chrome_options.add_argument("--incognito") # 無痕模式
chrome_options.add_argument("--disable-popup-blocking") # 停用 Chrome 的彈窗阻擋功能。
    
service = Service('./chromedriver.exe')  # chromedriver 路徑

# 建立 WebDriver 物件
driver = webdriver.Chrome(service=service, options=chrome_options)

# 進入目標網站
driver.get('https://www.twse.com.tw/zh/trading/historical/stock-day-avg.html')

stock_price_dict = defaultdict(dict)
stock_date_dict = defaultdict(dict)
stock_code_list = ['00934', '00935', '00936']
# stock_code_list = ['00934']


for stock_code in stock_code_list:    
    stock_input = driver.find_element(By.CLASS_NAME, "stock-code-autocomplete")
    stock_input.send_keys(stock_code)

    el = driver.find_element(By.NAME, "date")
    driver.execute_script("arguments[0].value='20251001';", el)

    # 定位按鈕並點擊
    button_element = driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/main/form/div/div[1]/div[3]/button")
    button_element.click()

    time.sleep(2)

    price_elements_list = driver.find_elements(By.CSS_SELECTOR, "tbody.is-last-page > tr")

    for element in price_elements_list:
        date_and_price = element.find_elements(By.TAG_NAME, 'td')

        date = date_and_price[0].text
        price = date_and_price[1].text
        if date == '月平均收盤價':
            continue

        stock_price_dict[stock_code][date] = price
        stock_date_dict[date][stock_code] = price
    
    stock_input.clear()

In [ ]:
print(dict(stock_date_dict))

In [ ]:
x = defaultdict(dict)

for date, price_dict in stock_date_dict.items():
    for k, v in price_dict.items():
        if x[k] == {}:
            x[k] = {
                "date": [date],
                "price": [v]
            }
        else:
            x[k]['date'].append(date)
            x[k]['price'].append(v)


In [2]:
import requests

url = "https://www.twse.com.tw/rwd/zh/afterTrading/STOCK_DAY_AVG?date=20250630&stockNo=0050&response=json&_=1721027771044"

response = requests.get(url)

print(response)
print(response.text)
data = response.json()

print(data)

<Response [200]>
{"stat":"OK","date":"20250630","title":"114年06月 0050 元大台灣50 日收盤價及月平均收盤價","fields":["日期","收盤價"],"data":[["114/06/02","175.90"],["114/06/03","177.15"],["114/06/04","181.30"],["114/06/05","181.65"],["114/06/06","181.95"],["114/06/09","183.70"],["114/06/10","188.65"],["114/06/18","47.57"],["114/06/19","47.10"],["114/06/20","47.03"],["114/06/23","46.64"],["114/06/24","47.49"],["114/06/25","48.18"],["114/06/26","48.24"],["114/06/27","48.29"],["114/06/30","48.36"],["月平均收盤價","106.20"]],"notes":["以上成交資料採市場交易時間之資料計算。","ETF證券代號第六碼為K、M、S、C者，表示該ETF以外幣交易。","權證證券代號可重複使用，權證顯示之名稱係目前存續權證之簡稱。"],"hints":"單位：元"}
{'stat': 'OK', 'date': '20250630', 'title': '114年06月 0050 元大台灣50 日收盤價及月平均收盤價', 'fields': ['日期', '收盤價'], 'data': [['114/06/02', '175.90'], ['114/06/03', '177.15'], ['114/06/04', '181.30'], ['114/06/05', '181.65'], ['114/06/06', '181.95'], ['114/06/09', '183.70'], ['114/06/10', '188.65'], ['114/06/18', '47.57'], ['114/06/19', '47.10'], ['114/06/20', '47.03'], ['114/06/23', '46.64'], 

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from datetime import datetime
from bokeh.palettes import Category10
import random
output_notebook()  # 若在 Jupyter Notebook 使用

p = figure(
    x_axis_type="datetime",
    width=600,
    height=400,
    title="股票價格折線圖"
)

for key, value in x.items():
    x[key]["date"] = [str(int(date[:3]) + 1911) + date[3:] for date in x[key]["date"]]

    x[key]["date"] = [datetime.strptime(date, "%Y/%m/%d") for date in x[key]["date"]]

    color = random.choice(Category10[10])  # 10 個漂亮顏色隨便挑

    p.line(x[key]["date"], x[key]["price"], line_width=2, color=color, legend_label=f"{key}")
    p.circle(x[key]["date"], x[key]["price"], size=6, color=color)


p.xaxis.axis_label = "日期"
p.yaxis.axis_label = "價格"

show(p)
